In [142]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

#關於第一個問題，個人覺得該被排除的欄位(也的確被排除)是名字、艙位及門票號，不只是因為這三項資料皆含有字串，更是因為這三項資料皆為亂數關係，
#即這三項是隨機排的，且更是否生存是孤立的，舉例來說，不會有人叫john就表示他有較高機率死漁船難，或者不會有人的傳票號為幾號或幾位數就容易身亡
#而艙位的部分在安排上也是隨機安排(頂多根據艙等安排)，補全資料上現在已不可補，所以刪掉省事

#由於載有缺失資料的欄位中只有年齡是完全數值的資料(登船處無法算平均，因此直接默認用眾數取代)，因此以年齡為例，其中平均數和中位數的大小差不太多
#因此所得的分數也相差不大，當然硬比的話第二個分數中位數比較好，第四個分數平均數比較好，這代表平均數和中位數間結果的準確度差不多，但是使用眾數
#時發現除了第2、3次外，眾數的表現優於平均數和中位數(第4次和平均數相當)，意味著用眾數填補後的平均表現會稍微優於平均數和中位數

#在測試針對「非數值型」的欄位，嘗試不同的處理策略比較結果時，對上一個問題遺留的條件先固定在使用中位數，然後分別測試性別使用onehotencoder，
#登船處使用labelencoder與兩者顛倒後，以及兩者皆用labelencoder做比較，結果看到兩者皆用labelencoder與皆用onehotencoder最後的結果並沒有差別
#或許是因為其編碼後的樣式差不多之原因，另外兩個結果基本上只有在部分次數上好於兩者方法相同的組合，整體上表現不超過兩者方法相同的組合

#再參考了課程提供了相關資料後，除了原本資料中推薦新增的特徵外，本人還加了是否為父子或母子的特徵，原因為因為船難發生時，有女性與小孩先走的慣例
#而小孩又不太可能單獨逃離，通常會有大人相伴，而考量到女性也可能單獨先走，因此增加母子時以"或"設定，而男性除非有小孩，不然無法先逃離，因此
#以"皆"設定，在新增一系列特徵後進行分析，發現其表現皆好於原本沒有新增特徵時的表現
pd.set_option('display.max_rows',None)
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna('S')

title = pd.DataFrame()
title['Title'] = df['Name'].map(lambda name: name.split(',')[1].split('.')[0].strip())

Title_Dictionary={
                  "Capt": "Officer",
                  "Col": "Officer",
                  "Major": "Officer",
                  "Jonkheer": "Royalty",
                  "Don": "Royalty",
                  "Sir": "Royalty",
                  "Dr": "Officer",
                  "Rev": "Officer",
                  "the Countess": "Royalty",
                  "Dona": "Royalty",
                  "Mme": "Mrs",
                  "Mile": "Miss",
                  "Ms": "Mrs",
                  "Mr": "Mr",
                  "Mrs": "Mrs",
                  "Miss": "Miss",
                  "Master": "Master",
                  "Lady": "Royalty"
}
title['Title'] = title.Title.map(Title_Dictionary)
title =pd.get_dummies(title.Title)

def cleanTicket(ticket):
    ticket =ticket.replace('.','')
    ticket = ticket.replace('/','')
    ticket = ticket.split()
    ticket = map(lambda t: t.strip(),ticket)
    ticket = list(filter(lambda t: not t.isdigit(), ticket))
    if len(ticket)>0:
        return ticket[0]
    else:
        return 'xxx'
ticket = pd.DataFrame()
ticket['Ticket'] = df['Ticket'].map(cleanTicket)
ticket = pd.get_dummies(ticket['Ticket'],prefix='Ticket')

df['ismotherchild'] = np.where((df['Age']<18) | (df['Parch'] > 0),1,0)
df['isfatherchild'] = np.where((df['Age']<18) & (df['Parch'] > 0),1,0)

family = pd.DataFrame()
family['FamilySize']=df['Parch']+df['SibSp']+1
family['Family_single']= family['FamilySize'].map(lambda s:1 if s ==1 else 0)
family['Family_small']=family['FamilySize'].map(lambda s:1 if 2 <=s <= 4 else 0)
family['Family_large']=family['FamilySize'].map(lambda s:1 if 5<= s else 0)

df = pd.get_dummies(df,columns = ['Sex','Embarked'])
df1 = pd.concat([df,title,ticket,family],axis=1)
df_train = df1.drop(['Cabin','Ticket','Name'],axis=1)






In [143]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[0.83240223 0.8258427  0.79775281 0.80898876 0.87078652]


In [98]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dfe2['Embarkedl']= le.fit_transform(dfe2['Embarked'])
dfe2 = dfe2.drop(['Embarked'],axis = 1)
dfe2['Sexl']= le.fit_transform(dfe2['Sex'])
dfe2 = dfe2.drop(['Sex'],axis = 1)
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']


train_X = df_train[columns_X]
train_y = df_train[columns_y]


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)

[0.78212291 0.78089888 0.78089888 0.76404494 0.81460674]


     PassengerId  Survived  Pclass    Age  SibSp  Parch      Fare  Sexl  \
0              1         0       3  22.00      1      0    7.2500     1   
1              2         1       1  38.00      1      0   71.2833     0   
2              3         1       3  26.00      0      0    7.9250     0   
3              4         1       1  35.00      1      0   53.1000     0   
4              5         0       3  35.00      0      0    8.0500     1   
5              6         0       3  28.00      0      0    8.4583     1   
6              7         0       1  54.00      0      0   51.8625     1   
7              8         0       3   2.00      3      1   21.0750     1   
8              9         1       3  27.00      0      2   11.1333     0   
9             10         1       2  14.00      1      0   30.0708     0   
10            11         1       3   4.00      1      1   16.7000     0   
11            12         1       1  58.00      0      0   26.5500     0   
12            13         